In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import io, math, os, sys
from base64 import b64decode
from pathlib import Path
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import numpy as np

import PIL

# Install daltonlens if necessary
try:
    from daltonlens import convert, simulate
except ImportError:
    !pip install -q daltonlens
    from daltonlens import convert, simulate
    
# Uncomment to get interactive plots.
# %matplotlib notebook

# Introduction

Goal is to generate the precomputed matrices / parameters needed by libDaltonLens or DaltonLens desktop.

# Brettel 1997 with sRGB

In [13]:
simulator = simulate.Simulator_Brettel1997(convert.LMSModel_sRGB_SmithPokorny75(), use_white_as_neutral=True)
simulator.dumpPrecomputedValues = True
np.set_printoptions(precision=4, suppress=True)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)


struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float rgbCvdFromRgb_2[9];

    // Normal of the separation plane to pick the right transform, already in the RGB space.
    // == normalInLms . lmsFromRgb
    float separationPlaneNormalInRgb[3];
};

static struct DLBrettel1997Params brettel_protan_params = {
    {
        0.14510, 1.20165, -0.34675,
        0.10447, 0.85316, 0.04237,
        0.00429, -0.00603, 1.00174,
    },
    {
        0.14115, 1.16782, -0.30897,
        0.10495, 0.85730, 0.03776,
        0.00431, -0.00586, 1.00155,
    },
    { 0.00048, 0.00416, -0.00464 }
};

struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float 

array([[[0, 0, 0]]], dtype=uint8)

# Viénot 1999 with sRGB

In [7]:
simulator = simulate.Simulator_Vienot1999(convert.LMSModel_sRGB_SmithPokorny75())
simulator.dumpPrecomputedValues = True
np.set_printoptions(precision=4, suppress=True)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)

static float vienot_protan_rgbCvd_from_rgb[] = {
    0.10889, 0.89111, -0.00000,
    0.10889, 0.89111, 0.00000,
    0.00447, -0.00447, 1.00000
};
static float vienot_deutan_rgbCvd_from_rgb[] = {
    0.29031, 0.70969, -0.00000,
    0.29031, 0.70969, -0.00000,
    -0.02197, 0.02197, 1.00000
};
static float vienot_tritan_rgbCvd_from_rgb[] = {
    1.00000, 0.15236, -0.15236,
    0.00000, 0.86717, 0.13283,
    -0.00000, 0.86717, 0.13283
};


array([[[0, 0, 0]]], dtype=uint8)

# Brettel with Vischeck parameters

In [8]:
simulator = simulate.Simulator_Vischeck()
simulator.dumpPrecomputedValues = True
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)

static float dl_brettel_protan_rgbCvd_from_rgb_1[] = {
    0.20611, 0.99192, -0.19803,
    0.16690, 0.79147, 0.04163,
    0.00967, -0.01209, 1.00241
};
static float dl_brettel_protan_rgbCvd_from_rgb_2[] = {
    0.20993, 1.01140, -0.22133,
    0.16609, 0.78738, 0.04653,
    0.00963, -0.01232, 1.00270
};
static float dl_brettel_protan_sepPlaneNormalInRgb[] = {
    0.00124, 0.00632, -0.00756
};
static float dl_brettel_deutan_rgbCvd_from_rgb_1[] = {
    0.43549, 0.72265, -0.15814,
    0.33522, 0.57087, 0.09391,
    -0.02269, 0.02905, 0.99364
};
static float dl_brettel_deutan_rgbCvd_from_rgb_2[] = {
    0.42922, 0.71316, -0.14238,
    0.33895, 0.57650, 0.08455,
    -0.02294, 0.02866, 0.99428
};
static float dl_brettel_deutan_sepPlaneNormalInRgb[] = {
    -0.00383, -0.00580, 0.00963
};
static float dl_brettel_tritan_rgbCvd_from_rgb_1[] = {
    1.01372, 0.08549, -0.09921,
    -0.02157, 0.86556, 0.15601,
    0.12159, 0.75769, 0.12072
};
static float dl_brettel_tritan_rgbCvd_from_rgb_2[] = {
  

array([[[0, 0, 0]]], dtype=uint8)